In [1]:
import numpy as np
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()
import openai
from datetime import datetime
import time

In [2]:
data_dir = '../data/'
output_dir = '../output/'
openai.api_key = os.environ["OPENAI_API_KEY"]

In [3]:
df = pd.read_excel(os.path.join(data_dir, 'new_prompts_manually_modified.xlsx'), sheet_name='Sheet1', header=0)
df = df.loc[~df['プロンプト'].str.contains('以下の項目'), :]
df = df.iloc[:, [0, 1, 2, 4]]
df_new_prompts = pd.read_excel(os.path.join(data_dir, 'プロンプト案20231124鈴木.xlsx'), sheet_name='Sheet1', header=0)

In [4]:
df['question'] = df_new_prompts['新プロンプト']

In [5]:
df.head()

,No,カテゴリ,質問項目,救急受診ガイドの判定,question
0,1,アルゴリズム,呼吸をしていない。息がない。,赤,救急受診が必要か判断してください。呼吸をしていない。息がない。
1,2,アルゴリズム,脈がない。心臓が止まっている。,赤,救急受診が必要か判断してください。脈がない。心臓が止まっている。
2,3,アルゴリズム,沈んでいる。,赤,救急受診が必要か判断してください。沈んでいる。
3,4,アルゴリズム,冷たくなっている。,赤,救急受診が必要か判断してください。冷たくなっている。
4,5,アルゴリズム,呼びかけても、反応がない。,赤,救急受診が必要か判断してください。呼びかけても、反応がない。


In [6]:
intro_child = '子供に'

In [7]:
def obtain_response(question: str):
    start_time = datetime.now()
    for _ in range(3):  # Retry up to 3 times
        try:
            response = openai.ChatCompletion.create(
                model='gpt-3.5-turbo',
                messages=[
                    {
                        'role': 'user',
                        'content': question
                    },
                ],
            )
            time_diff = round((datetime.now() - start_time).total_seconds())
            response_text = response.choices[0]['message']['content'].strip()
            return response_text, time_diff
        except openai.error.OpenAIError as e:
            print(f"OpenAI error: {e}")
            print(question)
            time.sleep(1)  # Wait for 1 second before retrying
        except Exception as e:
            print(f"Other error: {e}")
            time.sleep(1)  # Wait for 1 second before retrying
    else:
        print(f"Failed to get a response for question: {question}")
        return None, None

In [8]:
# Shuffle question order
#df_shuffled = df.sample(frac=1).reset_index(drop=True)

questions = []
numbers = []
responses = []
times = []

for index, row in df.iterrows():
    question = row['question']
    if '成人' in row['カテゴリ']:
        questions.append(question)
    elif '小児' in row['カテゴリ']:
        question = intro_child + question
        questions.append(question)
    else:
        questions.append(question)
    response, time_diff = obtain_response(question)
    numbers.append(row['No'])
    responses.append(response)
    times.append(time_diff)

df_temporary = pd.DataFrame({
    'No': numbers,
    'プロンプト': questions,
    '回答': responses,
    '回答時間(秒)': times
})

df_answer = df.merge(df_temporary, on='No')

In [ ]:
df_answer.to_excel(os.path.join(output_dir, 'ChatGPT_answer.xlsx'), engine='openpyxl', index=False)